# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xuewe\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\xuewe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xuewe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


c:\program files\python3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponses.db')
df = pd.read_sql_query('select * from messages', engine)

X = df['message'].values
Y = df.drop(['id','message','original','genre'],axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    wordTokens = word_tokenize(text)
    wordNetLemmatizer=WordNetLemmatizer()
    
    clean_tokens=[]
    for t in wordTokens:
        clean_token=wordNetLemmatizer.lemmatize(t).lower().strip()
        clean_tokens.append(clean_token)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([('vect',CountVectorizer(tokenizer=tokenize)),('tfidf',TfidfTransformer()),('clf',MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train,Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
Y_preds=pipeline.predict(X_test)

In [8]:
type(Y_preds)

numpy.ndarray

In [9]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [10]:
Y_preds[0]

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [11]:
type(Y_test.values)

numpy.ndarray

In [12]:
for i in range(len(Y.columns)):
    print("Label:", Y.columns[i])
    print(classification_report(Y_test.values[:,i], Y_preds[:,i]))

Label: related
             precision    recall  f1-score   support

          0       0.60      0.33      0.43      1253
          1       0.81      0.93      0.86      3952
          2       0.38      0.21      0.27        39

avg / total       0.76      0.78      0.76      5244

Label: request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      4358
          1       0.84      0.35      0.50       886

avg / total       0.88      0.88      0.86      5244

Label: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

Label: aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3100
          1       0.74      0.50      0.60      2144

avg / total       0.73      0.73      0.71      5244

Label: medical_help
     

c:\program files\python3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'tfidf__use_idf': [True, False],
        'tfidf__norm': ['l1', 'l2']
    }

    # optimize model
model = GridSearchCV(pipeline, param_grid=parameters,verbose=1)

In [14]:
model.fit(X_train, Y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 29.5min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'tfidf__use_idf': [True, False], 'tfidf__norm': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
Y_preds=model.predict(X_test)
for i in range(len(Y.columns)):
    print("Label:", Y.columns[i])
    print(classification_report(Y_test.values[:,i], Y_preds[:,i]))

Label: related
             precision    recall  f1-score   support

          0       0.65      0.33      0.43      1253
          1       0.81      0.94      0.87      3952
          2       0.33      0.18      0.23        39

avg / total       0.77      0.79      0.76      5244

Label: request
             precision    recall  f1-score   support

          0       0.88      0.99      0.93      4358
          1       0.83      0.35      0.49       886

avg / total       0.87      0.88      0.86      5244

Label: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      5220
          1       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5244

Label: aid_related
             precision    recall  f1-score   support

          0       0.72      0.88      0.79      3100
          1       0.75      0.49      0.59      2144

avg / total       0.73      0.72      0.71      5244

Label: medical_help
     

c:\program files\python3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [16]:
with open('models/model', 'wb') as outfile:
        pickle.dump(model, outfile)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.